# Neural plasticity is related to the behavior
## Chapter 1: check motor related potential (alpha band)
- preprocessing
    - create epoch for 20s/2s IMC, including EEG and EMG
    - identify bad chs
    - notch filter and bandpass
    - prepare epoch data, calculate muscle synergy for each subject (using var to decide the number), track explained variance
    - knn to cluster synergies and label them as 'ch_name'
    - create 2s epochs
- oscillatory analysis
- coherence analysis
    - alpha 4 FBC
    - CMC: alpha-activation
    - IMC: cosine similarity
## Notation:
- P1i: ipsilesional P1
- 34,35 are processed, redo prep on others 
## Calculate muscle synergy, and KNN identified common synergy
## whole brain cmc analysis

In [1]:
################## Import #####################
import os,mne,numpy as np, pandas as pd
from mne.preprocessing import ICA
from mne.time_frequency import tfr_morlet
import matplotlib.pyplot as plt
import seaborn as sns
from mne.time_frequency import tfr_morlet, psd_multitaper, psd_welch
from fooof import FOOOF
from fooof import FOOOFGroup
from fooof.objs import fit_fooof_3d, combine_fooofs
from mne.viz import plot_topomap
from mne.time_frequency import psd_welch
from matplotlib import cm, colors, colorbar

from fooof.bands import Bands
from fooof.analysis import get_band_peak_fg
from fooof.plts.spectra import plot_spectrum
%matplotlib qt
#############################################
########### function def
def emg2synergy(emg_data, emg_ch_names, show=True, n_components=3):
    '''
    parameters
    ----------
    emg_data: 2D numpy array (chs, time)
    emg_ch_names: list of str
    show: whether to visualize muscle synergy
    n_components: None or int. It should be greater than 2. Default 3. If specified, NMF will yield 3 muscle synergies. 
    If None, n_components = number of chs - 2 
    Notes
    -----
    We do not specify VAF_threhold and VAF_diff_threshold
    '''
    # build aligned_synergy_raw with muscle activation and synergic module factorization
    emg_data4muscleActivation = emg_data.copy()
    # bandpass to correct artifacts
    emg_data4muscleActivation = mne.filter.filter_data(emg_data4muscleActivation, sfreq=1000, l_freq=20, h_freq=400, method='iir')
    # demean
    emg_data4muscleActivation -= emg_data4muscleActivation.mean(axis=1)[:, None]
    emg_data4muscleActivation = np.abs(emg_data4muscleActivation)  # rectification
    emg_data4muscleActivation = emg_data4muscleActivation / np.max(emg_data4muscleActivation)  # normalization
    # emg_data2test /= emg_data2test.max(axis=1)[:, None] #  find max for each ch

    emg_data4muscleActivation = mne.filter.filter_data(emg_data4muscleActivation,
                                                       sfreq=1000, l_freq=None, h_freq=45, method='iir')  # low-pass smoothing,
    # please note that negative values appear in this step
    # muscle excitation (recursive filter)
    d = 10  # 10ms
    c1 = 0.5
    c2 = 0.5
    beta1 = c1 + c2
    beta2 = c1 * c2
    alpha = 1 + beta1 + beta2
    muscle_excitation = np.zeros(emg_data4muscleActivation.shape)
    for i in range(emg_data4muscleActivation.shape[1]-d):
        muscle_excitation[:, i+d] = alpha * emg_data4muscleActivation[:, i] - beta1*muscle_excitation[:, i+d-1] -\
        beta2 * muscle_excitation[:, i + d - 2]
    # muscle activation
    A = -1.5
    muscle_activation = (np.exp(A * muscle_excitation) - 1) / (np.exp(A) -1)
    if np.min(muscle_activation) < 0:  # handle negative values
        muscle_activation -= np.min(muscle_activation)

    # non-negative matrix factorization iterate from 2
    # stopping condition: as the n_components increase, the reconstruction error (Frobenius norm of the matrix difference) does not reduce significantly
    # In practice, the n_compoents stop increasing when VAF(n + 1) - VAF(n) < 0.05 or VAF(n) > 0.85
    VAF_diff = 1
    VAF_n = 0
    assert len(emg_ch_names) > 2, "must have more than 2 emg chs"
    if n_components is None:
        n_components = len(emg_ch_names) - 2
    n_components_init = 2
    VAF = []
    VAF_diff = []
    n_componentsInLoop = n_components_init
    while (n_componentsInLoop < (len(emg_ch_names) - 1)):
        model_n = NMF(n_components=n_componentsInLoop, init='random', random_state=0, max_iter=10000)  # max iteration is set to 10000
        model_n.fit_transform(muscle_activation)
        if n_componentsInLoop == n_components:
            muscle_synergy = model_n.fit_transform(muscle_activation)
            synergy_factor = model_n.components_   # the activation time course
        VAF_n = 1 - model_n.reconstruction_err_ / np.linalg.norm(muscle_activation)  # frobenius norm
        model_nPlus1 = NMF(n_components=n_componentsInLoop+1, init='random', random_state=0, max_iter=10000)
        model_nPlus1.fit_transform(muscle_activation)
        VAF_nPlus1 = 1 - model_nPlus1.reconstruction_err_ / np.linalg.norm(muscle_activation)
        VAF_diff_n = np.abs(VAF_n - VAF_nPlus1)
        VAF.append(VAF_n)
        VAF_diff.append(VAF_diff_n)
        n_componentsInLoop += 1
    df_muscle_synergy = pd.DataFrame(muscle_synergy.T, columns=emg_ch_names)
    if show:
        fig, axes = plt.subplots((n_components-1) // 4 + 1, 4, figsize=(24, 18))  # four cols
        if n_components > 4:
            for num_synergy in range(n_components):
                sns.barplot(ax=axes[num_synergy // 4, num_synergy % 4], x=df_muscle_synergy.columns, y=df_muscle_synergy.iloc[num_synergy].values,
                            color=sns.color_palette(n_colors=synergy_factor.shape[0])[num_synergy],
                            label='synergy_' + str(num_synergy))
                axes[num_synergy // 4, num_synergy % 4].set_title('synergic module' + str(num_synergy))
        else:
            for num_synergy in range(df_muscle_synergy.shape[0]):
                sns.barplot(ax=axes[num_synergy % 4], x=df_muscle_synergy.columns, y=df_muscle_synergy.iloc[num_synergy].values,
                            color=sns.color_palette(n_colors=synergy_factor.shape[0])[num_synergy],
                            label='synergy_' + str(num_synergy))
                axes[num_synergy % 4].set_title('synergic module' + str(num_synergy))
        plt.show()
    return muscle_synergy, synergy_factor, VAF, VAF_diff

def firstOnsetD(possibleOnsets):
    n=len(possibleOnsets)
    st_idx = 0
    onsets = []
    while st_idx < n-1:
        end = st_idx + 1
        dif = possibleOnsets[end] - possibleOnsets[st_idx]
        while end < n - 1 and possibleOnsets[end + 1] - possibleOnsets[end] == dif:
            end += 1
        onsets.append(possibleOnsets[st_idx])
        st_idx = end+1
    return onsets

def findOnsets(numberOfEpochs,raw_emg2cut,signature_chs_list,step=200,windowLen=3000,energy_q=0.99,energy_q_step=0.04,sfreq_emg=1000,onset_tuning=0):
    onsets = []
    try:
        while len(onsets) != numberOfEpochs:
            # pulling segmentation  - find best segmentation EMG-ch candidate
            signature_chsPower = [raw_emg2cut.to_data_frame()[ch].abs().sum() for ch in signature_chs_list]
            signature_ch = signature_chs_list[signature_chsPower.index(max(signature_chsPower))]
            df = raw_emg2cut.to_data_frame()[signature_ch].abs()
            emgEnergy = df.rolling(windowLen, win_type='boxcar').sum().dropna()[::step]
            possibleOnsets =np.array(emgEnergy[emgEnergy>emgEnergy.quantile(energy_q)].index.tolist())-windowLen
            onsets =np.array(firstOnsetD(possibleOnsets)) / sfreq_emg + onset_tuning  #related rto sampling rate of EMG, it is the time in seconds
            energy_q-=energy_q_step
    except:
        return False,onsets
#     print(len(onsets))
    return True,onsets
#############################

chs_map_rParetic = {'Fp1':'Fp1i','AF3':'AF3i','F3':'F3i','F7':'F7i','FC1':'FC1i','FC5':'FC5i','C3':'C3i',
                        'T7':'T7i','CP1':'CP1i','CP5':'CP5i','P3':'P3i','P7':'P7i','PO3':'PO3i','O1':'O1i',
                       'Fp2':'Fp1c','AF4':'AF3c','F4':'F3c','F8':'F7c','FC2':'FC1c','FC6':'FC5c','C4':'C3c',
                       'T8':'T7c','CP2':'CP1c','CP6':'CP5c','P4':'P3c','P8':'P7c','PO4':'PO3c','O2':'O1c'}
    
chs_map_lParetic = {'Fp2':'Fp1i','AF4':'AF3i','F4':'F3i','F8':'F7i','FC2':'FC1i','FC6':'FC5i','C4':'C3i',
                        'T8':'T7i','CP2':'CP1i','CP6':'CP5i','P4':'P3i','P8':'P7i','PO4':'PO3i','O2':'O1i',
                       'Fp1':'Fp1c','AF3':'AF3c','F3':'F3c','F7':'F7c','FC1':'FC1c','FC5':'FC5c','C3':'C3c',
                       'T7':'T7c','CP1':'CP1c','CP5':'CP5c','P3':'P3c','P7':'P7c','PO3':'PO3c','O1':'O1c'}

bad_eeg_chs = {'2':
               {'iMC':{'s01':['T8','C3','CP6'],'s02':[]},
                'iVC':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]}
               },
               '3':
               {'iMC':{'s01':[],'s02':[]},
                'iVC':{'s01':['T8','T7'],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]}
               },
                '4':
               {'iMC':{'s01':['Pz','T8'],'s02':['T8']},
                'iVC':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]}
               },
               '6':
               {'iMC':{'s01':[],'s02':[]},
                'iVC':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]}
               },
                '10':
               {'iMC':{'s01':['FC5c'],'s02':[]},
                'iVC':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]}
               },
               '14':
               {'iMC':{'s01':[],'s02':[]},
                'iVC':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]}
               },
               '15':
               {'iMC':{'s01':['T7c'],'s02':[]},
                'iVC':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]}
               },
               '16':
               {'iMC':{'s01':[],'s02':[]},
                'iVC':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]}
               },
               '17':
               {'iMC':{'s01':[],'s02':[]},
                'iVC':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]}
               },
               '18':
               {'iMC':{'s01':['F3c','Fp1c','AF3c'],'s02':[]},
                'iVC':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]}
               },
               '19':
               {'iMC':{'s01':['P7i', 'P7c'],'s02':[]},
                'iVC':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]}
               },
               '20':
               {'iMC':{'s01':[],'s02':[]},
                'iVC':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]}
               },
               '21':
               {'iMC':{'s01':['PO3c'],'s02':[]},
                'iVC':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]}
               },
               '24':
               {'iMC':{'s01':['FC5i','C3c'],'s02':[]},
                'iVC':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]}
               },
               '25':
               {'iMC':{'s01':['P7c'],'s02':[]},
                'iVC':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]}
               },
               '30':
               {'iMC':{'s01':['C3i', 'PO3i'],'s02':[]},
                'iVC':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]}
               },
               '32':
               {'iMC':{'s01':['F7i', 'T7c', 'Fz'],'s02':[]},
                'iVC':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]}
               },
               '34':
               {'iMC':{'s01':['PO3i', 'O1i'],'s02':['PO3','CP6','P3']},
                'iVC':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]}
               },
               '35':
               {'iMC':{'s01':['PO3i'],'s02':['PO3']},
                'iVC':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]}
               }
              }
ICs2remove = {'2':
               {'iMC':{'s01':[8],'s02':[]},
                'iVC_push':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'iVC_pull':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
               },
              '3':
               {'iMC':{'s01':[],'s02':[]},
                'iVC_push':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'iVC_pull':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
               },
                '4':
               {'iMC':{'s01':[0,2],'s02':[0]},
                'iVC_push':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'iVC_pull':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
               },
               '6':
               {'iMC':{'s01':[],'s02':[]},
                'iVC_push':{'s01':[0,5],'s02':[0,1,5],'s03':[0,3,12],'s04':[0,1,7],'s05':[0,1],'s06':[0,5]},
                'iVC_pull':{'s01':[0],'s02':[0,1],'s03':[0],'s04':[0,1],'s05':[0],'s06':[0]},
               },
                '10':
               {'iMC':{'s01':[0,2,3,7,15,17],'s02':[0,2,15,16,17]},
                'iVC_push':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'iVC_pull':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'srg':{'iMC':{'s01':[0],'s02':[0,4,7]}}
               },
              '14':
               {'iMC':{'s01':[0,1,2,5,6,8,12,14],'s02':[]},
                'iVC_push':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'iVC_pull':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'srg':{'iMC':{'s01':[0,1,15],'s02':[]}}
               },
              '15':
               {'iMC':{'s01':[0,1,2,3,7,11,15],'s02':[]},
                'iVC_push':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'iVC_pull':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'srg':{'iMC':{'s01':[0,1,2],'s02':[]}}
               },
              '16':
               {'iMC':{'s01':[],'s02':[]},
                'iVC_push':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'iVC_pull':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
               },
              '17':
               {'iMC':{'s01':[0],'s02':[]},
                'iVC_push':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'iVC_pull':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'srg':{'iMC':{'s01':[0,5],'s02':[]}}
               },
              '18':
               {'iMC':{'s01':[0,1,3],'s02':[]},
                'iVC_push':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'iVC_pull':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'srg':{'iMC':{'s01':[0],'s02':[]}}
               },
              '19':
               {'iMC':{'s01':[0,1,3],'s02':[]},
                'iVC_push':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'iVC_pull':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'srg':{'iMC':{'s01':[1],'s02':[]}}
               },
              '20':
               {'iMC':{'s01':[0,1,3,4,5,7,8,9,10,11,12,13,14,15,16],'s02':[4,10]},
                'iVC_push':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'iVC_pull':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'srg':{'iMC':{'s01':[0,2],'s02':[]}}
               },
              '21':
               {'iMC':{'s01':[2,3],'s02':[]},
                'iVC_push':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'iVC_pull':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'srg':{'iMC':{'s01':[3,7],'s02':[0,1,7,13]}}
               },
              '24':
               {'iMC':{'s01':[0,1,3,9,15],'s02':[0,5]},
                'iVC_push':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'iVC_pull':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'srg':{'iMC':{'s01':[0,2],'s02':[0,1,2,3,4]}}
               },      
              '25':
               {'iMC':{'s01':[0,1,5,7,10,12,13,15,17],'s02':[6]},
                'iVC_push':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'iVC_pull':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'srg':{'iMC':{'s01':[],'s02':[1]}}
               },
              '30':
               {'iMC':{'s01':[0,1,3,4],'s02':[0,4]},
                'iVC_push':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'iVC_pull':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'srg':{'iMC':{'s01':[1,5],'s02':[6]}}
               },
              '32':
               {'iMC':{'s01':[0,2,3,4,8,12],'s02':[1,7]},
                'iVC_push':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'iVC_pull':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'srg':{'iMC':{'s01':[0,2],'s02':[0,2]}}
               },
              '34':
               {'iMC':{'s01':[0,1,2,5,6,8,10,11,14],'s02':[0,5,11,12]},
                'iVC_push':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'iVC_pull':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'srg':{'iMC':{'s01':[0,6],'s02':[0]}}
               },
              '35':
               {'iMC':{'s01':[0,1,2,5,8,10,11,15],'s02':[0,1]},
                'iVC_push':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'iVC_pull':{'s01':[],'s02':[],'s03':[],'s04':[],'s05':[],'s06':[]},
                'srg':{'iMC':{'s01':[0,5],'s02':[0,3,4,6,9]}}
               }              
              }

iMC_onsets_dict = {'10': {'iMC':{'s01':[29.8, 71, 109, 147, 184.3],}},
                   '14':{'iMC':{'s01':[35,82,129.5,178,226],}},
                   '15': {'iMC':{'s01':[27, 75.5, 125, 174, 224],}},
                   '17': {'iMC':{'s01':[27.5, 76.5, 124.5, 174, 222.8],}},
                   '18': {'iMC':{'s01':[26.5,76,125.5,175,224],}},
                   '19': {'iMC':{'s01':[25,77,125,173,222],}},
                   '20':{'iMC':{'s01':[41.5, 90.5, 140, 189]}},
                   '21':{'iMC':{'s01':[37,87,136,185,234]}},
                   '24':{'iMC':{'s01':[37,87,136,185,234]}},
                   '25':{'iMC':{'s01':[32.5, 78, 125.5, 174.7, 223.7]}},
                   '30':{'iMC':{'s01':[38.5, 89.5, 138, 188]}},
                   '32':{'iMC':{'s01':[32.5,79.8,128,175.7,224.2]}},
                   '34':{'iMC':{'s01':[29,79,128,177,226.5]}},
                   '35':{'iMC':{'s01':[33.5,83,132.5,181,230]}}
                  }

chs_abnormal_linenoise_dict = {'10':{'iMC': {'s01':['O1i','O1c','F3c','Oz'],}},
                               '14':{'iMC':{'s01':['T7i','F7i', 'FC5i'],}},
                               '15':{'iMC': {'s01':['P3c', 'Pz'],}},
                               '17':{'iMC':{'s01':[],}},
                               '18':{'iMC':{'s01':['Cz', 'PO3c'],}},
                               '19':{'iMC':{'s01':['P3i'],}},
                               '20':{'iMC':{'s01':['FC5i', 'AF3i', 'F7i']}},
                               '21':{'iMC':{'s01':[]}},
                               '24':{'iMC':{'s01':['T7c', 'C3i', 'O1i']}},
                               '25':{'iMC':{'s01':[]}},
                               '30':{'iMC':{'s01':[]}},
                               '32':{'iMC':{'s01':[]}},
                               '34':{'iMC':{'s01':['Fp1i', 'F7i']}},
                               '35':{'iMC':{'s01':['Fp1c']}},
                      }

epochs2remove = {'10': {'iMC':{'s01':[],}},
                 '14':{'iMC':{'s01':[],}},
                 '15': {'iMC':{'s01':[0],}},
                 '17': {'iMC':{'s01':[],}},
                 '18': {'iMC':{'s01':[],}},
                 '19':{'iMC':{'s01':[2,3]}},
                 '20':{'iMC':{'s01':[]}},
                 '21':{'iMC':{'s01':[1,2,3]}},
                 '24':{'iMC':{'s01':[1,2,4]}},
                 '25':{'iMC':{'s01':[1,2]}},
                 '30':{'iMC':{'s01':[]}},
                 '32':{'iMC':{'s01':[4]}},
                 '34':{'iMC':{'s01':[]}},
                 '35':{'iMC':{'s01':[0,1]}},
                  }
    

## Filtering, identifying iMC onsets, chs with similar impedence, bad epochs
in this step, we remove DC and identify possible artifacts (only remove ocular and muscular one). delete noisy epoch and identify the bad channel


In [184]:
################## Customization ##################
# data_dir = 'D:/Data/MultiEEGEMG_stroke/'
data_dir = os.path.expanduser("/Users/ganshengt/Desktop/GT/gCMN/MultiEEGEMG_stroke")
subj_idx = '35'
contraction_type = 'iMC'
session_idx = 's01'
sfreq_emg=1000
sfreq_eeg=500
# session_duration = 45  # default is 50s
reject_criteria_eeg = dict(eeg=6e-4)       # 300 μV, do not exclude epochs containing ocular artifact
flat_criteria_eeg = dict(eeg=1e-6)           # 1 μV
manual_epoch = True
n_epochs = 5

################ fName
emg_fName = os.path.join(data_dir,'subj'+subj_idx,'EMG','subj'+subj_idx+'_'+contraction_type+'_'+session_idx+'.txt')
eeg_fName = os.path.join(data_dir,'subj'+subj_idx,'EEG','subj'+subj_idx+'_'+contraction_type+'_'+session_idx+'.set')
ica_dir = os.path.join(data_dir,'subj'+subj_idx,'ica')
epochs_f_ica_car_intpl_dir = os.path.join(data_dir,'subj'+subj_idx,'epochs_f_ica_car_intpl')
prep_report_dir = os.path.join(data_dir,'subj'+subj_idx,'prep_report')
if not os.path.exists(ica_dir):
    os.makedirs(ica_dir)
if not os.path.exists(epochs_f_ica_car_intpl_dir):
    os.makedirs(epochs_f_ica_car_intpl_dir)
if not os.path.exists(prep_report_dir):
    os.makedirs(prep_report_dir)
# ica_fName = os.path.join(data_dir,'subj'+subj_idx,'ica','subj'+subj_idx+'_'+contraction_type+'_'+session_idx+'_ica.fif')
epochs_f_ica_car_intpl_fName = os.path.join(epochs_f_ica_car_intpl_dir, 'subj' + subj_idx + '_' + contraction_type + '_' + session_idx + '_f_ica_car_intpl_epo.fif')
alignmentInfo_fName = os.path.join(data_dir,'subj'+subj_idx,'subj'+subj_idx+'_alignmentInfo.txt')
alignmentInfo = pd.read_csv(alignmentInfo_fName, skiprows=0,sep = ',',engine = 'python')
subjInfo_fName = os.path.join(data_dir,'subj_info.txt')
subjInfo = pd.read_csv(subjInfo_fName, skiprows=0,
                       sep = ',',engine = 'python') 
affected_h = subjInfo[subjInfo['subj_idx']==int(subj_idx)]['affected_h'].values[0]
############################################################

####### eeg reading ###########
raw_eeg = mne.io.read_raw_eeglab(eeg_fName,preload=True)


# electrode projection according to the affected hand
if affected_h == 'l':
    mne.channels.rename_channels(raw_eeg.info, chs_map_lParetic)
elif affected_h == 'r':
    mne.channels.rename_channels(raw_eeg.info, chs_map_rParetic)

#     mne.rename_channels(raw_eeg.info, chs_map_rParetic)
else:
    print('check info file')

    # ipsilesional always on the left brain
enobio_montage = mne.channels.read_custom_montage(os.path.join(data_dir,'Enobio32_l_paretic.locs'))    
raw_eeg.set_montage(enobio_montage)

# alignment
events,events_id = mne.events_from_annotations (raw_eeg)
iMC_start = events[0][0] - alignmentInfo.loc[(alignmentInfo['sessionIdx']==session_idx) & 
                                      (alignmentInfo['contraction_type']==contraction_type),
                                      'EEG'].values[0]  #data point
raw_eeg.info['bads']=bad_eeg_chs[subj_idx][contraction_type][session_idx]

raw_eeg.crop(tmin = alignmentInfo.loc[(alignmentInfo['sessionIdx']==session_idx) & 
                                      (alignmentInfo['contraction_type']==contraction_type),
                                      'EEG'].values[0]/raw_eeg.info['sfreq'])

# find chs with similar impedence
raw_eeg_copy = raw_eeg.copy()
raw_eeg_copy.filter(l_freq=45,h_freq=55)
chs_noise_level = np.trapz(np.abs(raw_eeg_copy.get_data()), axis=1)
sns.set_theme(style="whitegrid")
df_chs_noise_level = pd.DataFrame(np.expand_dims(chs_noise_level, 0), columns=raw_eeg_copy.info['ch_names'])

fig = plt.figure(figsize=(16,8))
ax = sns.barplot(data=df_chs_noise_level)
plt.axhline(y=df_chs_noise_level.mean().mean(), color='r', linestyle='-')
plt.axhline(y=df_chs_noise_level.mean().mean()+ 2*df_chs_noise_level.mean().std(), color='b', linestyle='-')
plt.axhline(y=df_chs_noise_level.mean().mean()- 2*df_chs_noise_level.mean().std(), color='b', linestyle='-')
fig.savefig(os.path.join(prep_report_dir, 'subj' + subj_idx + '_' + contraction_type + '_' + session_idx + '_line_noise_level.eps'))

# filter
raw_eeg.filter(l_freq=0.5,h_freq=45, method='iir')
raw_eeg.notch_filter(freqs=np.arange(50, 201, 50))

############# emg reading ##############
ch_types = ['emg']*8
ch_names = ['FDS', 'FCU', 'FCR', 'ECU', 'ECRL', 'BBS', 'TBL', 'LD']
emg_data = pd.read_csv(emg_fName, header = None,skiprows=3, 
                       sep = ' ',usecols=np.arange(0,8),skipfooter=0,engine = 'python')
############### alignment 
emg_data= emg_data.iloc[alignmentInfo.loc[(alignmentInfo['sessionIdx']==session_idx) & 
                                      (alignmentInfo['contraction_type']==contraction_type),
                                      'EMG'].values[0]:,:].values
############## band-pass filter
emg_data=emg_data.T/1e6
emg_data = mne.filter.filter_data(emg_data,sfreq=1000,l_freq=10,h_freq=200)
info = mne.create_info(ch_names=ch_names, sfreq=sfreq_emg, ch_types=ch_types)
raw_emg = mne.io.RawArray(emg_data, info)

# integration
raw_eeg_2intergrate = raw_eeg.copy()
raw_emg_2intergrate = raw_emg.copy()
raw_emg_2intergrate.resample(sfreq=sfreq_eeg)
hybrid_tmax = min(raw_eeg.get_data().shape[1],raw_emg_2intergrate.get_data().shape[1]) / sfreq_eeg - 1
raw_emg_2intergrate.crop(tmax =hybrid_tmax)
raw_eeg_2intergrate.crop(tmax =hybrid_tmax)
raw_emg_2intergrate.info['highpass'] = raw_eeg_2intergrate.info['highpass']
raw_emg_2intergrate.info['lowpass'] = raw_eeg_2intergrate.info['lowpass']
# events, events_id = mne.events_from_annotations(raw_eeg)
raw_hybrid = raw_eeg_2intergrate.add_channels([raw_emg_2intergrate])
fig = raw_eeg.plot(title='raw_eeg', n_channels=16, duration=40, scalings=dict(eeg=5e-5))
fig.savefig(os.path.join(prep_report_dir, 'subj' + subj_idx + '_' + contraction_type + '_' + session_idx + '_raw_eeg.eps'))

# epoch

# iMC_onsets = np.linspace(start = events[0][0]/raw_eeg.info['sfreq'], 
#                          stop = events[0][0]/raw_eeg.info['sfreq']+ session_duration*4, num=5) - raw_eeg.first_time
# manual edition
if manual_epoch:
    iMC_onsets = iMC_onsets_dict[subj_idx][contraction_type][session_idx]
else:
    foundOnsets, iMC_onsets = findOnsets(5,raw_emg,['FCR'],step=1000, windowLen=3000, energy_q_step=0.03, sfreq_emg=1000)
    if foundOnsets:
        print(iMC_onsets)
    else:
        print('fail in identifying onsets')

durations = [0.] * n_epochs
descriptions = ['onset_iMC'] * n_epochs
annot = mne.Annotations(onset=iMC_onsets, duration=durations,
                                    description=descriptions,
                                    orig_time=raw_eeg.info['meas_date'])

raw_hybrid.set_annotations(annot)
fig = raw_hybrid.plot(title='raw_hybrid', start=50, n_channels=40, duration=40, scalings=dict(eeg=4e-5, emg=5e-4))
fig.savefig(os.path.join(prep_report_dir, 'subj' + subj_idx + '_' + contraction_type + '_' + session_idx + '_raw_hybrid.eps'))

Used Annotations descriptions: ['1', 'empty']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 45 - 55 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 45.00
- Lower transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Upper passband edge: 55.00 Hz
- Upper transition bandwidth: 13.75 Hz (-6 dB cutoff frequency: 61.88 Hz)
- Filter length: 147 samples (0.294 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.50, 45.00 Hz: -6.02, -6.02 dB

Setting up band-stop filter

FIR filter parameters
---------------------

/var/folders/7q/rjvcsg0s39b9750fsn4_8ylh0000gr/T/ipykernel_70453/4202559382.py:106: DeprecationWarning: highpass cannot be set directly. Please use methods inst.filter() instead. This warning will turn into an error after 0.24
  raw_emg_2intergrate.info['highpass'] = raw_eeg_2intergrate.info['highpass']
/var/folders/7q/rjvcsg0s39b9750fsn4_8ylh0000gr/T/ipykernel_70453/4202559382.py:107: DeprecationWarning: lowpass cannot be set directly. Please use method inst.filter() instead. This warning will turn into an error after 0.24
  raw_emg_2intergrate.info['lowpass'] = raw_eeg_2intergrate.info['lowpass']
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially tra

## epoch data and ICA

In [187]:
events,events_id = mne.events_from_annotations (raw_hybrid)
epochs_hybrid = mne.Epochs(raw_hybrid,events,tmin=-15.0,tmax=15.0,baseline=None,preload=True)

# remove noisy epochs
epochs_hybrid.drop(epochs2remove[subj_idx][contraction_type][session_idx])
fig = epochs_hybrid.plot(n_epochs=1, title='epochs_hybrid')
fig.savefig(os.path.join(prep_report_dir, 'subj' + subj_idx + '_' + contraction_type + '_' + session_idx + '_epoch_hybrid.eps'))

# ICA
ica = ICA(n_components = 0.99,random_state=97)
ica.fit(epochs_hybrid)
fig = ica.plot_sources(epochs_hybrid,stop=1)
fig.savefig(os.path.join(prep_report_dir, 'subj' + subj_idx + '_' + contraction_type + '_' + session_idx + '_ica_source.eps'))
fig = ica.plot_components(inst = epochs_hybrid)
fig[0].savefig(os.path.join(prep_report_dir, 'subj' + subj_idx + '_' + contraction_type + '_' + session_idx + '_ica_comp_.jpg'))

Used Annotations descriptions: ['onset_iMC']
Not setting metadata
Not setting metadata
5 matching events found
No baseline correction applied
0 projection items activated
Loading data for 5 events and 15001 original time points ...
0 bad epochs dropped
Dropped 2 epochs: 0, 1


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


Fitting ICA to data using 31 channels (please be patient, this may take a while)
Selecting by explained variance: 16 components
Fitting ICA took 0.4s.
Not setting metadata
Not setting metadata
3 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
Not setting metadata
3 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
Not setting metadata
3 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
Not setting metadata
3 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
Not setting metadata
3 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
Not setting metadata
3 matching events found
No baseline correction applied
0 projecti

## Remove artifactual ICs
- remove EOG, ECG, EMG, and fluctuation with great amplitude, NOTE that widespread EMG, ECG can be removed with CAR
- use bad ch and remove epoch to remove ch-localized/temporal localized artifacts

In [189]:
# remove ICs
ica.exclude = ICs2remove[subj_idx][contraction_type][session_idx]
for ic in ica.exclude:
    fig = ica.plot_properties(inst = epochs_hybrid, picks=ic) 
    fig[0].savefig(os.path.join(prep_report_dir, 'subj' + subj_idx + '_' + contraction_type + '_' + session_idx + '_ica_component_' + str(ic) + '.jpg'))
epochs_hybrid_ica = epochs_hybrid.copy()
ica.apply(epochs_hybrid_ica)
fig = epochs_hybrid_ica.plot(n_epochs=1,title='after ica')
fig.savefig(os.path.join(prep_report_dir, 'subj' + subj_idx + '_' + contraction_type + '_' + session_idx + '_epochs_afterICA.eps'))

    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
Not setting metadata
3 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
Not setting metadata
3 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
Not setting metadata
3 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
Not setting metadata
3 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
Not setting metadata
3 matching events found
No baseline correction applied
0 projecti

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


## Common average reference

In [190]:
epochs_hybrid_ica_car = epochs_hybrid_ica.copy()
for ch in chs_abnormal_linenoise_dict[subj_idx][contraction_type][session_idx]:
    if ch not in epochs_hybrid_ica_car.info['bads']:
        epochs_hybrid_ica_car.info['bads'].append(ch)
epochs_hybrid_ica_car, ref_data = mne.set_eeg_reference(epochs_hybrid_ica_car, ref_channels=list(np.setdiff1d(epochs_hybrid_ica_car.info['ch_names'][:32],
                                                                                                  epochs_hybrid_ica_car.info['bads'])))
fig = epochs_hybrid_ica_car.plot(title='car', n_epochs=1)
fig.savefig(os.path.join(prep_report_dir, 'subj' + subj_idx + '_' + contraction_type + '_' + session_idx + '_epochs_afterCAR.eps'))
# mne.set_eeg_reference(epochs_hybrid_prep, ref_channels=['AF3', 'AF4', 'C3', 'C4', 'CP1', 'CP2', 'CP5', 'CP6', 'Cz', 'F3', 'F4'])

EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


## Interpolation

In [191]:
epochs_hybrid_ica_car_intpl = epochs_hybrid_ica_car.interpolate_bads()
fig = epochs_hybrid_ica_car_intpl.plot(n_epochs=1, title='interpolated')
fig.savefig(os.path.join(prep_report_dir, 'subj' + subj_idx + '_' + contraction_type + '_' + session_idx + '_epochs_afterITPL.eps'))

Interpolating bad channels
    Automatic origin fit: head of radius 95.0 mm
Computing interpolation matrix from 30 sensor positions
Interpolating 2 sensors


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


## PSD for specified ch and all chs

In [192]:
ch_name = 'C3i'
epo_idx = 1
kwargs = dict(fmin=1, fmax=40, n_jobs=1)
psds_welch_mean_baseline, freqs_mean_baseline = psd_welch(epochs_hybrid_ica_car_intpl.copy().crop(tmin=-10,tmax=-1), average='mean', n_fft=512, **kwargs)
psds_welch_mean_motor, freqs_mean_motor = psd_welch(epochs_hybrid_ica_car_intpl.copy().crop(tmin=0,tmax=9), average='mean', n_fft=512, **kwargs)

# Convert power to dB scale.
psds_welch_mean_baseline = 10 * np.log10(psds_welch_mean_baseline * 1e12)
psds_welch_mean_motor = 10 * np.log10(psds_welch_mean_motor * 1e12)

ch_idx = epochs_hybrid_ica_car_intpl.info['ch_names'].index(ch_name)
_, ax = plt.subplots()
ax.plot(freqs_mean_baseline, psds_welch_mean_baseline[epo_idx, ch_idx, :], color='k',
        ls='--', label='baseline')
ax.plot(freqs_mean_motor, psds_welch_mean_motor[epo_idx, ch_idx, :], color='k',
        ls='-', label='motor')
ax.set(title='Welch PSD ({}, Epoch {})'.format(ch_name, epo_idx),
       xlabel='Frequency (Hz)', ylabel='Power Spectral Density (dB, V^2/Hz)')
ax.legend(loc='upper right')
plt.show()
fig=ax.get_figure()
fig.savefig(os.path.join(prep_report_dir, 'subj' + subj_idx + '_' + contraction_type + '_' + session_idx + '_' + str(ch_name) + '_epoch' + str(epo_idx) + '_welch_psd.eps'))

# across trials
psds_welch_mean_baseline_across_trials = np.mean(psds_welch_mean_baseline, axis=0)
psds_welch_mean_motor_across_trials = np.mean(psds_welch_mean_motor, axis=0)

_, ax = plt.subplots()
ax.plot(freqs_mean_baseline, psds_welch_mean_baseline_across_trials[ch_idx, :], color='k',
        ls='--', label='baseline')
ax.plot(freqs_mean_motor, psds_welch_mean_motor_across_trials[ch_idx, :], color='k',
        ls='-', label='motor')

ax.set(title='Welch PSD ({})'.format(ch_name),
       xlabel='Frequency (Hz)', ylabel='Power Spectral Density (dB)')
ax.legend(loc='upper right')
plt.show()
fig=ax.get_figure()
fig.savefig(os.path.join(prep_report_dir, 'subj' + subj_idx + '_' + contraction_type + '_' + session_idx + '_' + str(ch_name) + '_all_epochs_welch_psd.eps'))

fig = epochs_hybrid_ica_car_intpl.plot_psd(fmin=1., fmax=40., tmin=-10, tmax=-1, average=True, normalization='full')
fig.legend(['all chs, baseline'])
fig.savefig(os.path.join(prep_report_dir, 'subj' + subj_idx + '_' + contraction_type + '_' + session_idx + '_all_chs_baseline_multitaper_psd.eps'))

fig = epochs_hybrid_ica_car_intpl.plot_psd(fmin=1., fmax=40., tmin=0, tmax=10, average=True, normalization='full')
fig.legend(['all chs, motor'])
fig.savefig(os.path.join(prep_report_dir, 'subj' + subj_idx + '_' + contraction_type + '_' + session_idx + '_all_chs_motor_multitaper_psd.eps'))
# epochs_hybrid_prep_car_intpl.plot_psd_topomap(ch_type='eeg', normalize=False)
# power.plot_topo(baseline=(-0.5, 0), mode='logratio', title='Average power')

Effective window size : 1.024 (s)
Effective window size : 1.024 (s)


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


    Using multitaper spectrum estimation with 7 DPSS windows


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


    Using multitaper spectrum estimation with 7 DPSS windows


The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


## topograph of bands (morlet)

In [3]:
freqs = np.logspace(*np.log10([4, 30]), num=20)
baseline = (-6, -1)
n_cycles = freqs/2 # different number of cycle per frequency
fig = plt.figure(figsize=(8, 7))
power, itc = tfr_morlet(epochs_hybrid.copy().crop(tmin=-6,tmax=4), freqs=freqs, n_cycles=n_cycles, use_fft=True,
                        return_itc=True, decim=3, n_jobs=1)
power.plot_topo(baseline=baseline, mode='logratio', title='Average power')

fig, axis = plt.subplots(1, 3, figsize=(12, 5))
# power.plot_topomap(ch_type='eeg', tmin=0, tmax=10, fmin=1, fmax=4,
#                    baseline=baseline, mode='logratio', axes=axis[0][0],
#                    title='Delta', show=False)
power.plot_topomap(ch_type='eeg', tmin=-1, tmax=4, fmin=4, fmax=8, 
                   baseline=baseline, mode='logratio', axes=axis[0],
                   title='Theta (motor-baseline)', show=False)
power.plot_topomap(ch_type='eeg', tmin=-1, tmax=4, fmin=8, fmax=12,
                   baseline=baseline, mode='logratio', axes=axis[1],
                   title='Alpha (motor-baseline)', show=False)
power.plot_topomap(ch_type='eeg', tmin=-1, tmax=4, fmin=13, fmax=30,
                   baseline=baseline, mode='logratio', axes=axis[2],
                   title='Beta (motor-baseline)', show=False)
# fig.savefig(os.path.join(prep_report_dir, 'subj' + subj_idx + '_' + contraction_type + '_' + session_idx + '_tf_band_topo.jpg'))

NameError: name 'epochs_hybrid' is not defined

## Save epochs for further analyses

In [194]:
epochs_hybrid_ica_car_intpl.save(epochs_f_ica_car_intpl_fName,overwrite=True)

## Oscillation analysis

### Information to keep
- power spectrum for each electrode, each trial (welch psd)
- fitting features including center frequency, power, bandwidth of peaks, offset, knee and exponent of the aperiodic component

### data format would be

| subj | task | session |  ch |   epoch   |   trial   | freq_range |  spectrum  | center_freq | power | bandwidth | offset |  exponent | R2 | error |
| ---- | ---- | ------- | --- | --------- | --------- | ---------- | ---------- | ----------- | ----- | --------- | ------ |  -------- | -- | ----- |
|.  20 | imc  | pre_TMS | C3i |  averaged | motor.    |  \[list\]  |  \[list\]  |   \[list\]  | list. | list.     |  list  |   lits.   |    |.      |

### Knee is not considered in the fitting


In [195]:
fg = FOOOFGroup(peak_width_limits=[1, 6], min_peak_height=0.15, peak_threshold=2, max_n_peaks=4, verbose=False, aperiodic_mode='fixed')
freq_range = [2,40]
kwargs = dict(fmin=1, fmax=40, n_jobs=1)

dict_2df = {'subj':[],
           'task':[],
           'session':[],
           'ch':[],
           'trial':[],
           'freq_range':[],
           'spectrum':[],
           'center_freq':[],
           'power':[],
           'bandwidth':[],
           'offset':[],
           'exponent':[],
           'R2':[],
           'error':[]}

bands = Bands({'theta': [3, 7],
               'alpha': [8, 12],
               'beta': [13, 30]})
ax_x_start = 0.95
ax_x_width = 0.02
ax_y_start = 0.1
ax_y_height = 0.7
# for ch in epochs_hybrid_ica_car_intpl.info['ch_names'][:32]:
psds_welch_baseline, freqs_baseline = psd_welch(epochs_hybrid_ica_car_intpl.copy().crop(tmin=-11,tmax=-1), average='mean', n_fft=512, **kwargs)
psds_welch_motor, freqs_motor = psd_welch(epochs_hybrid_ica_car_intpl.copy().crop(tmin=-1,tmax=9), average='mean', n_fft=512, **kwargs)
# psd: epoch * ch * spect
psds_welch_baseline_averaged_over_trials = np.mean(psds_welch_baseline * 1e12, axis=0)  # change unit to V2/Hz  
psds_welch_motor_averaged_over_trials = np.mean(psds_welch_motor * 1e12, axis=0)
for trial in ['baseline', 'motor']:
    exec('fg.fit(freqs_' + trial + ', psds_welch_' + trial + '_averaged_over_trials, freq_range)')

    fig, ax = plt.subplots(ncols=3, figsize=(12, 4), gridspec_kw=dict(top=0.9),
                           sharex=True, sharey=True)

    thetas = get_band_peak_fg(fg, bands.theta)
    alphas = get_band_peak_fg(fg, bands.alpha)
    betas = get_band_peak_fg(fg, bands.beta)

    # Extract the power values from the detected peaks
    exec('theta_pw_' + trial + '= np.nan_to_num(thetas[:, 1])')
    exec('alpha_pw_' + trial + '= np.nan_to_num(alphas[:, 1])')
    exec('beta_pw_' + trial + '= np.nan_to_num(betas[:, 1])')
    exec('plot_topomap(theta_pw_' + trial + ', raw_eeg.info, axes=ax[0], show=False)')
    exec('plot_topomap(alpha_pw_' + trial + ', raw_eeg.info, axes=ax[1], show=False)')
    exec('im, cm = plot_topomap(beta_pw_' + trial + ', raw_eeg.info, axes=ax[2], show=False)')
    ax[0].set_title('Theta', fontweight='bold')
    ax[1].set_title('Alpha', fontweight='bold')
    ax[2].set_title('Beta', fontweight='bold')

    cbar_ax = fig.add_axes([ax_x_start, ax_y_start, ax_x_width, ax_y_height])
    clb = fig.colorbar(im, cax=cbar_ax)
    clb.ax.set_title('peak power',fontsize=20)
    fig.savefig(os.path.join(prep_report_dir, 'subj' + subj_idx + '_' + contraction_type + '_' + session_idx + trial + '_pw_topo.jpg'))
    # build feature df
    for ch_result_idx in range(len(fg.group_results)):
        n_peaks = fg.group_results[ch_result_idx].peak_params.shape[0]
        for peak_idx in range(n_peaks):
            dict_2df['ch'].append(epochs_hybrid_ica_car_intpl.info['ch_names'][ch_result_idx])
            dict_2df['trial'].append(trial)
            dict_2df['spectrum'].append(fg.power_spectra[ch_result_idx])
            dict_2df['center_freq'].append(fg.group_results[ch_result_idx].peak_params[peak_idx][0])
            dict_2df['power'].append(fg.group_results[ch_result_idx].peak_params[peak_idx][1])
            dict_2df['bandwidth'].append(fg.group_results[ch_result_idx].peak_params[peak_idx][2])
            dict_2df['offset'].append(fg.group_results[ch_result_idx].aperiodic_params[0])
            dict_2df['exponent'].append(fg.group_results[ch_result_idx].aperiodic_params[1])
            dict_2df['R2'].append(fg.group_results[ch_result_idx].r_squared)
            dict_2df['error'].append(fg.group_results[ch_result_idx].error)

dict_2df['subj'] = [str(subj_idx)] * len(dict_2df['ch'])
dict_2df['task'] = ['imc'] * len(dict_2df['ch'])
dict_2df['session'] = ['pre_TMS'] * len(dict_2df['ch'])
dict_2df['freq_range'] = [freqs_baseline] * len(dict_2df['ch'])

fig, ax = plt.subplots(ncols=3, figsize=(12, 4), gridspec_kw=dict(top=0.9),
                       sharex=True, sharey=True)
plot_topomap(theta_pw_motor - theta_pw_baseline, raw_eeg.info, axes=ax[0], show=False)
plot_topomap(alpha_pw_motor - alpha_pw_baseline, raw_eeg.info, axes=ax[1], show=False)
im, cm = plot_topomap(beta_pw_motor - beta_pw_baseline, raw_eeg.info, axes=ax[2], show=False)

ax[0].set_title('Theta', fontweight='bold')
ax[1].set_title('Alpha', fontweight='bold')
ax[2].set_title('Beta', fontweight='bold')
cbar_ax = fig.add_axes([ax_x_start, ax_y_start, ax_x_width, ax_y_height])
clb = fig.colorbar(im, cax=cbar_ax)
clb.ax.set_title('peak power (motor-baseline)',fontsize=20)
fig.savefig(os.path.join(prep_report_dir, 'subj' + subj_idx + '_' + contraction_type + '_' + session_idx + 'motor_baseline_pw_topo.jpg'))
fig.savefig(os.path.join(prep_report_dir, 'subj' + subj_idx + '_' + contraction_type + '_' + session_idx + 'motor_baseline_pw_topo.eps'))

df = pd.DataFrame(dict_2df)
df.to_csv(os.path.join(prep_report_dir, 'subj' + subj_idx + '_' + contraction_type + '_' + session_idx + 'psd_fit.csv'))

Effective window size : 1.024 (s)
Effective window size : 1.024 (s)


/Users/ganshengt/opt/anaconda3/envs/p38/lib/python3.8/site-packages/fooof/objs/group.py:378: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  out = np.array([np.insert(getattr(data, name), 3, index, axis=1)
/Users/ganshengt/opt/anaconda3/envs/p38/lib/python3.8/site-packages/fooof/objs/group.py:378: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  out = np.array([np.insert(getattr(data, name), 3, index, axis=1)
/var/folders/7q/rjvcsg0s39b9750fsn4_8ylh0000gr/T/ipykernel_70453/1810489079.py:78: RuntimeWarning: More than 20 figures have been opened. Figures cr

In [196]:
fg.report()

                                                                                                  
                                       FOOOF - GROUP RESULTS                                      
                                                                                                  
                             Number of power spectra in the Group: 32                             
                                                                                                  
                        The model was run on the frequency range 2 - 40 Hz                        
                                 Frequency Resolution is 0.98 Hz                                  
                                                                                                  
                              Power spectra were fit without a knee.                              
                                                                                                  
          

/Users/ganshengt/opt/anaconda3/envs/p38/lib/python3.8/site-packages/fooof/objs/group.py:378: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  out = np.array([np.insert(getattr(data, name), 3, index, axis=1)


## Chapter 2: synergy calculation
### Section 2-1: import data

In [ ]:
epochs_hybrid_ica_car_intpl = mne.read_epochs(epochs_f_ica_car_intpl_fName)

In [ ]:
'''
The length of baseline segment is 18s, we generate 30 3s epochs with a step of 0.5s
'''
srg_tmin = 0.0
srg_tmax = 18.001
length_epochs = 3
n_srg_epochs = 15
sliding_step = 1
length_rect_epochs = 16
n_srg_rect_epochs = 10
sliding_step_rect = 0.2

# create epochs_srg #
motorPeriods = ['', '_rect']
for motorPeriod in motorPeriods:
    srg_raw = aligned_raw.copy().crop(tmin=srg_tmin, tmax=srg_tmax)
    exec('srg' + motorPeriod + '_onsets = np.arange(0, n_srg' + motorPeriod + '_epochs * sliding_step' + motorPeriod + ', sliding_step' + motorPeriod + ')')
    exec('durations = [0.] * len(' + 'srg' + motorPeriod + '_onsets)')
    descriptions = ['srgOnset'] * len(durations)
    exec('annot_srg = mne.Annotations(onset=srg' + motorPeriod + '_onsets, duration=durations,\
                            description=descriptions, orig_time=aligned_raw.info["meas_date"])')
    srg_raw.set_annotations(annot_srg)
    events, events_id = mne.events_from_annotations(srg_raw)
    # No rejection occurred
    exec('srg' + motorPeriod + '_epochs = mne.Epochs(srg_raw, events, tmin=0.0, tmax=length' + motorPeriod + '_epochs, baseline=None, preload=True)')
    exec('srg' + motorPeriod + '_epochs.save(srg' + motorPeriod + '_epochs_fName, overwrite=True)')
    
    # calculate muscle synergy epochs
    exec('srg_onsets_samplePoints = sfreq_emg * srg' + motorPeriod + '_onsets')
    exec('synergy_factor' + motorPeriod + '_epochs_data = []')
    n_components = 4
    muscle_synergy_data = []
    VAF2df = []
    VAF_diff2df = []
    for onset in srg_onsets_samplePoints:
        # 3002 -> 1501 for 3s
        exec('muscle_synergy, synergy_factor, VAF, VAF_diff = emg2synergy(emg_data[:, int(onset):int(onset+length' + motorPeriod +
             '_epochs * sfreq_emg + 2)], emg_ch_names=emg_ch_names, show=False, n_components=n_components)')  # 2get the same length
        exec('synergy_factor' + motorPeriod + '_epochs_data.append(synergy_factor)')
        muscle_synergy_data.append(muscle_synergy)
        VAF2df.append(VAF)
        VAF_diff2df.append(VAF_diff)
        
    muscle_synergy_data2df = np.zeros((len(srg_onsets_samplePoints) * n_components, len(emg_ch_names) + 2))
    for n_synergy in range(n_components):
        for n_epoch in range(len(srg_onsets_samplePoints)):
            muscle_synergy_data2df[(n_epoch) * n_components + n_synergy, :] = np.hstack([muscle_synergy_data[n_epoch].T[n_synergy],
                                                                                        [n_synergy + 1, n_epoch]])
    df_muscle_synergy = pd.DataFrame(data=muscle_synergy_data2df, columns=emg_ch_names + ['n_synergy', 'n_epoch'])
    exec('df_muscle_synergy.to_csv(df_srg' + motorPeriod + '_muscle_synergy_fName)')
         
    VAF_data2df = np.zeros((len(VAF2df), 2 * np.shape(VAF2df)[1] + 1))
    for n_compoent in range(np.shape(VAF2df)[1]):   # range of n_components based on which NMF was fitted
        for n_epoch in range(np.shape(VAF2df)[0]):  # number of epochs
            VAF_data2df[n_epoch, :] = np.hstack([VAF2df[n_epoch], VAF_diff2df[n_epoch], [n_epoch]])
    df_VAF = pd.DataFrame(data=VAF_data2df, columns=['VAF_'+str(i+2) for i in range(np.shape(VAF2df)[1])] +
                          ['VAF_diff_'+str(i+2) for i in range(np.shape(VAF2df)[1])] + ['n_epoch'])  # n starts from 2
    exec('df_VAF.to_csv(df_srg' + motorPeriod + '_VAF_fName)')
    
    # visual inspect noisy epochs (extreme participant movements)
    # create synergy epochs
    exec('ch_types = ["emg"] * np.shape(synergy_factor' + motorPeriod + '_epochs_data)[1]')  # number of synergies
    exec('synergy_ch_names = ["synergy_" + str(i+1) for i in range(np.shape(synergy_factor' + motorPeriod +'_epochs_data)[1])]')
    info = mne.create_info(ch_names=synergy_ch_names, sfreq=sfreq_emg, ch_types=ch_types)
    exec('srg' + motorPeriod + '_synergy_epochs = srg' + motorPeriod + '_epochs.copy()')
    exec('srg' + motorPeriod + '_synergy_epochs.drop_channels(emg_ch_names)')
    exec('synergy_epochs = mne.EpochsArray(synergy_factor' + motorPeriod + '_epochs_data, info=info).resample(sfreq=sfreq_eeg)')
    exec('synergy_epochs.info["highpass"] = srg' + motorPeriod + '_synergy_epochs.info["highpass"]')
    exec('synergy_epochs.info["lowpass"] = srg' + motorPeriod + '_synergy_epochs.info["lowpass"]')
    exec('srg' + motorPeriod + '_synergy_epochs.add_channels([synergy_epochs])')  # eeg-synergy epochs 
    exec('srg' + motorPeriod + '_synergy_epochs.save(srg' + motorPeriod + '_synergy_epochs_fName, overwrite=True)')
    exec('srg' + motorPeriod + '_synergy_epochs.plot(scalings=dict(eeg=1e-4, emg=srg' + motorPeriod +
         '_synergy_epochs.get_data().max()/2), picks="all", title="srg' + motorPeriod + '_synergy_epochs")')  # all epochs are plotted at once

